***Notebook with EEG describe features and spectrogram describe features,
    feel free to fork this and do leave an upvote if you find this helpful :')***

In [ ]:
import pandas as pd
import copy
from sklearn.model_selection import train_test_split

import numpy as np
import keras
import os

import lightgbm as lgb
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [ ]:
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
sub = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv')

# Making x_train

In [ ]:
# %%time
# import os
# import concurrent.futures
# import pandas as pd

# folder_path = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms'

# def process_file(filename):
#     file_path = os.path.join(folder_path, filename)
#     df = pd.read_parquet(file_path)
#     df = df.describe().drop(df.describe().index[0])
#     return filename.split('.')[0], df.to_numpy().flatten()

# train_spectrograms = {}
# file_list = os.listdir(folder_path)

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     futures = {executor.submit(process_file, filename): filename for filename in file_list}
#     for future in concurrent.futures.as_completed(futures):
#         filename = futures[future]
#         try:
#             result = future.result()
#             train_spectrograms[result[0]] = result[1]
#         except Exception as e:
#             print(f"Error processing file {filename}: {e}")

In [ ]:
from joblib import dump, load
# dump(train_eegs,'train_eegs.joblib')
train_spectrograms = load('/kaggle/input/spects/train_spectrograms.joblib')

In [ ]:
x_train = train[['eeg_id','spectrogram_id','seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote','expert_consensus']]
x_train = x_train.drop_duplicates()

In [ ]:
# dict of eeg_id and set of its unique spectrograms
eeg_set = {}
for i in x_train[['eeg_id','spectrogram_id']].values:
    if i[0] in eeg_set:
        if i[1] not in eeg_set[i[0]]:
            eeg_set[i[0]] = eeg_set.get(i[0]).add(i[1])
    else:
        temp = set({})
        temp.add(i[1])
        eeg_set[i[0]] = temp

In [ ]:
#dict of eeg_id and its merged spectrogram_
eeg_spect = {}
for i in list(eeg_set.keys()):
    #j is spects for a 'i' eeg_id key
    for j in list(eeg_set[i]):
        if i in eeg_spect:
            eeg_spect[i] = eeg_spect[i].append(train_spectrograms[str(j)])
        else:
            temp = []
            temp.append(train_spectrograms[str(j)])
            eeg_spect[i] = temp
            
for i in list(eeg_spect.keys()):
    eeg_spect[i] = np.asarray(eeg_spect[i])

In [ ]:
val = []
for i in x_train.values:
    key = i[0]
    val.append(eeg_spect[key])
val = np.asarray(val)
val = val.reshape(-1,2807)

In [ ]:
n = pd.DataFrame(val, columns= range(len(val[0])))

In [ ]:
x_train = x_train.reset_index()
x_train = pd.concat([x_train, n], axis=1)
x_train = x_train.drop(columns = ['index'])

In [ ]:
train_eegs = load('/kaggle/input/train-eegs/train_eegs.joblib')

val = []
for i in x_train.values:
    key = str(i[0])
    val.append(train_eegs[key])
val = np.asarray(val)

In [ ]:
n = pd.DataFrame(val, columns= range(3000,3000+len(val[0])) )

x_train = x_train.reset_index()
x_train = pd.concat([x_train, n], axis=1)
x_train = x_train.drop(columns = ['index'])

# Making x_test

In [ ]:
x_test = test[['eeg_id']]
x_sub = sub[['eeg_id']]

In [ ]:
%%time
import os
import concurrent.futures
import pandas as pd

folder_path = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms'

def process_file(filename):
    file_path = os.path.join(folder_path, filename)
    df = pd.read_parquet(file_path)
    df = df.describe().drop(df.describe().index[0])
    return filename.split('.')[0], df.to_numpy().flatten()

test_spectrograms = {}
file_list = os.listdir(folder_path)

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = {executor.submit(process_file, filename): filename for filename in file_list}
    for future in concurrent.futures.as_completed(futures):
        filename = futures[future]
        try:
            result = future.result()
            test_spectrograms[result[0]] = result[1]
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

In [ ]:
# dict of eeg_id and set of its unique containing spects
eeg_set = {}
for i in test[['eeg_id','spectrogram_id']].values:
    if i[0] in eeg_set:
        if i[1] not in eeg_set[i[0]]:
            eeg_set[i[0]] = eeg_set.get(i[0]).add(i[1])
    else:
        temp = set({})
        temp.add(i[1])
        eeg_set[i[0]] = temp

In [ ]:
#dict of eeg_id and its merged spectrogram_ids
eeg_spect = {}
for i in list(eeg_set.keys()):
    #j is spectrogram_id in 'i' - eeg_id key
    for j in list(eeg_set[i]):
        if i in eeg_spect:
            eeg_spect[i] = eeg_spect[i].append(test_spectrograms[str(j)])
        else:
            temp = []
            temp.append(test_spectrograms[str(j)])
            eeg_spect[i] = temp
            
for i in list(eeg_spect.keys()):
    eeg_spect[i] = np.asarray(eeg_spect[i])
    if eeg_spect[i].shape[0] >1:
        eeg_spect[i] = np.mean(eeg_spect[i], axis=0)
        

In [ ]:
val = []
for i in x_sub.values:
    key = i[0]
    val.append(eeg_spect[key])
val = np.asarray(val)
val = val.reshape(-1,2807)

In [ ]:
n = pd.DataFrame(val, columns= range(len(val[0])) )

In [ ]:
x_sub = x_sub.reset_index()
x_sub = pd.concat([x_sub, n], axis=1)
x_sub = x_sub.drop(columns= ['index'])

In [ ]:
%%time
import os
import concurrent.futures
import pandas as pd

folder_path = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs'

def process_file(filename):
    file_path = os.path.join(folder_path, filename)
    df = pd.read_parquet(file_path)
    df = df.describe().drop(df.describe().index[0])
    return filename.split('.')[0], df.to_numpy().flatten()

test_eegs = {}
file_list = os.listdir(folder_path)

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = {executor.submit(process_file, filename): filename for filename in file_list}
    for future in concurrent.futures.as_completed(futures):
        filename = futures[future]
        try:
            result = future.result()
            test_eegs[result[0]] = result[1]
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

In [ ]:
val = []
for i in x_sub.values:
    key = str(int(i[0]))
    val.append(test_eegs[key])
val = np.asarray(val)

In [ ]:
n = pd.DataFrame(val, columns= range(3000,3000+len(val[0])) )

x_sub = x_sub.reset_index()
x_sub = pd.concat([x_sub, n], axis=1)
x_sub = x_sub.drop(columns= ['index'])

# data labeling and preprocessing

In [ ]:
x_train['expert_consensus'] = x_train['expert_consensus'].replace({'Seizure': 0, 'LPD': 1, 'GPD': 2,'LRDA':3, 'GRDA':4, 'Other':5 })

In [ ]:
lgb_train = x_train.drop(columns = ['eeg_id','spectrogram_id','seizure_vote','lpd_vote',	'gpd_vote',	'lrda_vote','grda_vote','other_vote','expert_consensus']).copy()
lgb_test = x_sub.drop(columns = ['eeg_id']).copy()

# LGBM Modelling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
lgb_train = pd.DataFrame(scaler.fit_transform(lgb_train), columns=lgb_train.columns)
lgb_test = pd.DataFrame(scaler.transform(lgb_test), columns=lgb_test.columns)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss


params = {
    'objective': 'multiclass',
    'num_class': 6,
    'boosting_type': 'gbdt',
    'metric': 'multi_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'max_depth': 17,
    'min_data_in_leaf': 20,
    'device': 'gpu',
}


lgb_model = lgb.LGBMClassifier(**params)
lgb_model.fit(lgb_train, x_train.expert_consensus , verbose=0)

y_pred = lgb_model.predict(lgb_test)
y_pred_proba = lgb_model.predict_proba(lgb_test)

In [ ]:
sub['seizure_vote'] = y_pred_proba[:,0]
sub['lpd_vote'] = y_pred_proba[:,1]
sub['gpd_vote'] = y_pred_proba[:,2]
sub['lrda_vote'] = y_pred_proba[:,3]
sub['grda_vote'] = y_pred_proba[:,4]
sub['other_vote'] = y_pred_proba[:,5]

In [ ]:
sub.to_csv('submission.csv',index = False)